In [41]:
import re,yaml,codecs
from os.path import expanduser
import pandas as pd
from cellbell import ding

In [42]:
home = expanduser("~")
annee="2020"
numGroupe=50
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/"+annee
serie=repertoire+"/K%02d/"%numGroupe

In [43]:
nomLexique=home+"/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/bdlexique.txt"
dfLexique=pd.read_csv(nomLexique,encoding="utf8",sep=";")

In [44]:
dfLexique.columns="ortho phono liaison cat ms tam lemme L23 F1 F2 F3 F4".split()
catLexique={"NOM":"N G".split(),"VER":"V".split(),"ADJ":"F G J K L M".split()}
tam="pi ii ai".split()
pers="3S 3P".split()

In [45]:
#dfLexique#[dfLexique["lemme"]=="coyote"]["ortho"].tolist()

In [46]:
newStems={}
with open(serie+"Stems.csv", 'r') as stream:
    stems=yaml.safe_load(stream)
stems

{'ADJ': {'amn': [u'\xe9norme'],
  'inn': [u'm\xe9tamorphos\xe9'],
  'jeb': ['grand'],
  'jki': ['trois'],
  'kvi': ['inconnu'],
  'lof': ['folle'],
  'nib': ['triste'],
  'noj': ['jaune'],
  'nti': ['plusieurs'],
  'oup': ['petit'],
  'ovn': ['nouveau'],
  'rid': [u'diff\xe9rent'],
  'riv': ['quatre'],
  'roiu': ['deux'],
  'rum': ['plusieurs'],
  'tbi': ['petit'],
  'tik': ['atypique'],
  'vel': ['beau'],
  'wan': ['orange'],
  'zib': [u'\xe9trange']},
 'NOM': {'A': {'aze': [u'plan\xe8te'],
   'ber': ['chevache'],
   'bikar': ['bouc-cerf'],
   'ela': ['Ela'],
   'gipi': ['autreille'],
   'gira': ['gorille'],
   'kiri': ['chaien'],
   'lim': ['lirat'],
   'mor': ['mouchphant'],
   'nira': ['jungle'],
   'peka': ['liane'],
   'upou': ['animal'],
   'urio': ['conscience'],
   'uviu': ['falaise'],
   'wor': ['loubis']},
  'H': {'amam': [u'Am\xe8le'],
   'diro': ['fille'],
   'fred': ['Fred'],
   'kalaba': ['kalaba'],
   'mahira': ['Mari'],
   'mavi': ['Mavy'],
   'meme': [u'M\xe9lo\xe9'],

In [47]:
def getFormesVerbe(verbe):
    p3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="pi")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    p3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="pi")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    i3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ii")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    i3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ii")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    a3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ai")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    a3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ai")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    
    return p3S,p3P,i3S,i3P,a3S,a3P
                             
def getFormes(lemme,cat):
    if cat=="VER":
        flexion=list(getFormesVerbe(lemme))
    elif cat=="DET":
        if lemme=="DEF": flexion="le la les l".split()
        elif lemme=="IND": flexion="un une des dE d".split()
        else: flexion="ce cet cette ces".split()
    else:
        flexion=dfLexique.loc[dfLexique["cat"].isin(catLexique[cat]) & (dfLexique["lemme"]==lemme), "ortho"].tolist()
        flexion=list(set(flexion))
        if cat=="NOM" and lemme.endswith("s"):
            print "attention",lemme,"a été modifié avec un S"
            lemme=lemme[:-1]+"S"
        if lemme in flexion:
            flexion.remove(lemme)
    return [lemme]+flexion

In [48]:
apiSymboles=u"ʃʒŋθð"
sampaSymboles=u"SZNTD"
translit={}
for i,s in enumerate(apiSymboles):
    translit[s]=sampaSymboles[i]
translit

{u'\xf0': u'D',
 u'\u014b': u'N',
 u'\u0283': u'S',
 u'\u0292': u'Z',
 u'\u03b8': u'T'}

In [49]:
def getTranslit(chaine):
    result=chaine
    for s in translit:
        result=result.replace(s,translit[s])
    return result

In [50]:
getTranslit(u"ʃaʒuŋoθið")

u'SaZuNoTiD'

In [51]:
for cat in stems:
    if cat not in newStems:
        newStems[cat]={}
    if cat not in ["PREP"]:
        print cat
        for element in stems[cat]:
            if isinstance(stems[cat][element],dict):
                newStems[cat][element]={}
                for word in stems[cat][element]:
                    if isinstance(stems[cat][element][word],dict):
                        subelement=word
                        if subelement not in newStems[cat][element]:
                            newStems[cat][element][subelement]={}
                        for w in stems[cat][element][subelement]:
                            mot=getTranslit(w)
                            flexion=stems[cat][element][subelement][w]
                            lemme=flexion[0]
                            print getFormes(lemme,cat)
                            newStems[cat][element][subelement][mot]=getFormes(lemme,cat)                            
                    else:
                        mot=getTranslit(word)
                        flexion=stems[cat][element][word]
                        lemme=flexion[0]
                        print lemme, cat
                        print getFormes(lemme,cat)
                        newStems[cat][element][mot]=getFormes(lemme,cat)
                    
            else:
                flexion=stems[cat][element]
                lemme=flexion[0]
                print lemme
                print getFormes(lemme,cat)
                mot=getTranslit(element)
                newStems[cat][mot]=getFormes(lemme,cat)        
    else:
        print cat
        for element in stems[cat]:
            mot=getTranslit(element)
            newStems[cat][mot]=stems[cat][element]

ADJ
jaune
['jaune', u'jaunes']
énorme
[u'\xe9norme', u'\xe9normes']
triste
['triste', u'tristes']
deux
['deux']
inconnu
['inconnu', u'inconnue', u'inconnues', u'inconnus']
plusieurs
['plusieurs']
métamorphosé
[u'm\xe9tamorphos\xe9']
trois
['trois']
différent
[u'diff\xe9rent', u'diff\xe9rentes', u'diff\xe9rente', u'diff\xe9rents']
petit
['petit', u'petite', u'petites', u'petits']
nouveau
['nouveau', u'nouvelle', u'nouvelles', u'nouvel', u'nouveaux']
orange
['orange']
folle
['folle']
plusieurs
['plusieurs']
grand
['grand', u'grands', u'grande', u'grandes']
quatre
['quatre']
étrange
[u'\xe9trange', u'\xe9tranges']
atypique
['atypique', u'atypiques']
petit
['petit', u'petite', u'petites', u'petits']
beau
['beau', u'beaux', u'belles', u'bel', u'belle']
NOM
conscience NOM
['conscience', u'consciences']
animal NOM
['animal']
autreille NOM
['autreille']
mouchphant NOM
['mouchphant']
Ela NOM
['Ela']
gorille NOM
['gorille', u'gorilles']
planète NOM
[u'plan\xe8te', u'plan\xe8tes']
bouc-cerf NOM
[

In [52]:
newStems

{'ADJ': {u'amn': [u'\xe9norme', u'\xe9normes'],
  u'inn': [u'm\xe9tamorphos\xe9'],
  u'jeb': ['grand', u'grands', u'grande', u'grandes'],
  u'jki': ['trois'],
  u'kvi': ['inconnu', u'inconnue', u'inconnues', u'inconnus'],
  u'lof': ['folle'],
  u'nib': ['triste', u'tristes'],
  u'noj': ['jaune', u'jaunes'],
  u'nti': ['plusieurs'],
  u'oup': ['petit', u'petite', u'petites', u'petits'],
  u'ovn': ['nouveau', u'nouvelle', u'nouvelles', u'nouvel', u'nouveaux'],
  u'rid': [u'diff\xe9rent',
   u'diff\xe9rentes',
   u'diff\xe9rente',
   u'diff\xe9rents'],
  u'riv': ['quatre'],
  u'roiu': ['deux'],
  u'rum': ['plusieurs'],
  u'tbi': ['petit', u'petite', u'petites', u'petits'],
  u'tik': ['atypique', u'atypiques'],
  u'vel': ['beau', u'beaux', u'belles', u'bel', u'belle'],
  u'wan': ['orange'],
  u'zib': [u'\xe9trange', u'\xe9tranges']},
 'NOM': {'A': {u'aze': [u'plan\xe8te', u'plan\xe8tes'],
   u'ber': ['chevache'],
   u'bikar': ['bouc-cerf'],
   u'ela': ['Ela'],
   u'gipi': ['autreille'],
  

In [53]:
with open(serie+"Stems.yaml", 'w') as stream:
    yaml.safe_dump(newStems,stream=stream,default_flow_style=False,allow_unicode=True)

In [54]:
ding()